In [1]:
!pip install konlpy
from konlpy.tag import Komoran
import requests
import lxml.html
import cssselect as external_cssselect
from lxml.cssselect import CSSSelector
import tqdm
import time

You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

day_minus_day = datetime.now()-relativedelta(days =8) # 오늘 날짜 -1 
datetime = day_minus_day.strftime("%Y%m%d")    #문자로 반환

day_minus_month = day_minus_day -relativedelta(months=1)
month_ago =  day_minus_month.strftime("%Y%m%d")#strftime() 은 지정된 포맷에 맞춰 date 객체의 정보를 문자열로 반환한다. month_ago는 오늘날짜의 한달 전


url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=STC&cluster=y&q=화재원인&period=y&sd='+ month_ago +'000001&ed=' +  datetime +'235959'
url


'https://search.daum.net/search?nil_suggest=btn&w=news&DA=STC&cluster=y&q=화재원인&period=y&sd=20190701000001&ed=20190801235959'

In [3]:
urls = [] 
for page in range(1, 3): 
    res = requests.get(url.format(page)) 
    root = lxml.html.fromstring(res.text) 
    for link in root.cssselect('a.f_nb'): 
         urls.append(link.attrib['href'])
len(urls)

22

In [4]:
articles = [] 
i=0

for u in tqdm.tqdm_notebook(urls): 
    if not u.startswith('http'): 
        continue
    try :
        res = requests.get(u) 
        time.sleep(1)
        print(i ,' ',res.status_code)
        i = i+1
        root = lxml.html.fromstring(res.text) 
        body = root.cssselect('.article_view').pop() 
        content = body.text_content().strip() # 본문을 가져와 앞뒤 공백을 제거
        articles.append(content) 
    except :
        pass


0   200
1   200
2   200
3   200
4   200
5   200
6   200
7   200
8   200
9   200
10   200
11   200
12   200
13   200
14   200
15   200
16   200
17   200
18   200
19   200
20   200
21   200



In [5]:
import pandas
article_list = pandas.DataFrame({'article': articles}) #변수명수정하기
# df.to_excel('daum_news_ai.xlsx')
article_list.head()

,article
0,오늘(28일) 아침 7시 15분쯤 경기도 포천시에 있는 군부대 보급창고에서 불이 났...
1,오늘 아침 7시 15분쯤 경기도 포천시에 있는 군부대 보급창고에서 불이 났습니다. ...
2,오늘(31일) 새벽 2시 반쯤 부산 기장군에 있는 3층짜리 한의원 건물 2층에서 원...
3,부산서 패널구조 건물 3개 동 화재 부산경찰청 제공\n \n ...
4,광주 광산구 신창동 한 유통센터에서 화재가 발생해 소방당국이 진화 작업을 벌이고 있...


In [6]:
from konlpy.tag import Twitter #+코모란 Komoran 이게 좀 좋다
tagger = Twitter()

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\user\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
def kor_noun(text): 
    words = [] 
    for w in tagger.nouns(text): 
        if len(w) > 1: 
             words.append(w) 
    return words


In [10]:
import pandas as pd


cv = CountVectorizer(tokenizer=kor_noun, max_features=1000)
articles = article_list['article'].astype(str)
tdm = cv.fit_transform(articles)
words = cv.get_feature_names()
imageKeyword=sorted(list(zip(tdm.sum(axis=0).flat, words)), reverse=True)[:100] #이거 바꾸기 
pd.DataFrame(imageKeyword).to_excel('imageKeyword.xlsx')

In [11]:
keyword_20190807=list(pd.DataFrame(imageKeyword).iloc[[17,39,51,87],1]) #수작업해야 할 필요 있음
keyword_20190807

['중고차', '폐차', '배터리', '펌프']

In [12]:
def make_folder(work_dir):
    if not os.path.isdir(work_dir):
        os.mkdir(work_dir)
    else:
        os.chdir(work_dir)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os
import urllib
import argparse
import time
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
root_dir = os.getcwd()    # 기본 디렉토리로 이동 (어떤 컴퓨터든) 
work_dir =  root_dir + "\\" + today
final_dir = work_dir + "\\search_issue_month"


#찾고자 하는 검색어를 url로 만들어 준다.
for corekeyword in keyword_20190807:
    
    
    
    searchterm = corekeyword
    url = "https://www.google.co.in/search?q="+searchterm+"&source=lnms&tbm=isch"
    # chrom webdriver 사용하여 브라우저를 가져온다.
    browser = webdriver.Chrome('C:/chromedriver')
    browser.get(url)

    # User-Agent를 통해 봇이 아닌 유저정보라는 것을 위해 사용
    header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}
    # 이미지 카운트 (이미지 저장할 때 사용하기 위해서)
    counter = 0
    succounter = 0
   
            

    os.chdir(root_dir)
    
    if not os.path.isdir(work_dir):
        os.mkdir(work_dir)
    elif os.path.isdir(work_dir):
        os.chdir(work_dir)  
     
    os.chdir(work_dir) 

    if not os.path.exists("search_issue_month"):
        os.mkdir("search_issue_month")        
    
    os.chdir(final_dir)     
    
    # 크롤링을 하는 날짜로 만든 폴더명에 접속, 이 폴더에 '검색어' 폴더가 없으면 만들어준다.(이미지 저장 폴더를 위해서)
    if not os.path.exists(searchterm):
        os.mkdir(searchterm)

    for _ in range(200):
        browser.execute_script("window.scrollBy(0,10000)")

    for x in browser.find_elements_by_xpath('//div[contains(@class,"rg_meta")]'): # div태그에서 class name이 rg_meta인 것을 찾아온다
        counter = counter + 1
        print("Total Count:", counter)
        print("Succsessful Count:", succounter)
        print("URL:",json.loads(x.get_attribute('innerHTML'))["ou"])

        # 이미지 url
        img = json.loads(x.get_attribute('innerHTML'))["ou"]
        # 이미지 확장자
        imgtype = json.loads(x.get_attribute('innerHTML'))["ity"]

        # 구글 이미지를 읽고 저장한다.
        try:
            #req = urllib.request(img, headers={'User-Agent': header})
            raw_img = urllib.request.urlopen(img).read()
            File = open(os.path.join(searchterm , searchterm + "_" + str(counter) + "." + imgtype), "wb")
            File.write(raw_img)
            File.close()
            succounter = succounter + 1
        except:
                print("can't get img")
    os.chdir(root_dir) #다시 root_dir로 돌아오기.
    print(succounter, "pictures succesfully downloaded")

Total Count: 1
Succsessful Count: 0
URL: http://media1.or.kr/wp-content/uploads/2019/04/%ED%81%AC%EA%B8%B0%EB%B3%80%ED%99%98KakaoTalk_20190422_203421687.jpg
Total Count: 2
Succsessful Count: 1
URL: http://www.autohub.co.kr/img/main/banner.jpg
Total Count: 3
Succsessful Count: 2
URL: http://autoimg.danawa.com/html/images/buy/usedcar_directauto_01.jpg
can't get img
Total Count: 4
Succsessful Count: 2
URL: https://post-phinf.pstatic.net/MjAxNzA4MzBfMzMg/MDAxNTA0MDU4Mzc4NjA1.mDIuWrTFloJZFqk6vFrxV9bXtpCYdavy_zDEETYwSmEg.yhF9JKAJGR1t6sbPhCFYh_ykwJv6ae1JGn_CtVN0ZTAg.PNG/image_3945280531504058325313.png?type=w1200
Total Count: 5
Succsessful Count: 3
URL: https://post-phinf.pstatic.net/MjAxNzEwMDVfODQg/MDAxNTA3MTk3MTg5OTky.yUpuUQnqP7G6X-_eIIpneFiJ0lIsIsXu-4QskxJDkOcg.b43pp5m-OJmBtOQx_jyLZDoN9bHrfEAf6LXB9dQvsVkg.JPEG/%EB%B2%A8%EB%A1%9C%EC%8A%A4%ED%84%B0-tile_%EB%B3%B5%EC%82%AC.jpg?type=w1200
Total Count: 6
Succsessful Count: 4
URL: http://img.hani.co.kr/imgdb/resize/2014/0204/00479032901_2014020

Succsessful Count: 61
URL: http://certifiedcar.hyundaicapital.com/hcscert/HCL8073_OG_1_SQ_.JPG
Total Count: 65
Succsessful Count: 62
URL: http://image.chosun.com/sitedata/image/201903/27/2019032702345_0.jpg
Total Count: 66
Succsessful Count: 63
URL: http://easylaw.go.kr/CSA/template/2017/07/10/csp1000[20170710111826246].jpg
Total Count: 67
Succsessful Count: 64
URL: http://t1.daumcdn.net/liveboard/pickplus/950d36d41d344c8c894a34f077de515a.JPG
Total Count: 68
Succsessful Count: 65
URL: https://www-trucknbus.hyundai.com/kr/upload/news/BL00000424/2018-Certification-System-b.jpg
Total Count: 69
Succsessful Count: 66
URL: https://pds.joins.com//news/component/htmlphoto_mmdata/201808/07/92b702c5-1c41-4b33-be92-0df5c74e0ff7.jpg
Total Count: 70
Succsessful Count: 67
URL: https://t1.daumcdn.net/cfile/tistory/264CF242556EC28025
Total Count: 71
Succsessful Count: 68
URL: http://www.autodaily.co.kr/news/photo/201810/405570_31751_5154.jpg
Total Count: 72
Succsessful Count: 69
URL: https://t1.daumcd

URL: http://www.hansungusedcar.co.kr/images/etc/visual_authUsedCar.jpg
Total Count: 133
Succsessful Count: 128
URL: http://www.gangnam.go.kr/upload/editor/2019/05/14/7df38476-7cc4-4d64-9f56-de25c16639a6.jpg
Total Count: 134
Succsessful Count: 129
URL: http://www.sbiztoday.kr/news/photo/201803/1198_1159_3635.jpg
Total Count: 135
Succsessful Count: 130
URL: http://file.newswire.co.kr/data/datafile2/thumb_480/2018/08/31015275_20180831140619_6187086798.jpg
Total Count: 136
Succsessful Count: 131
URL: https://post-phinf.pstatic.net/MjAxODEwMDVfMjUx/MDAxNTM4NzI2NDkxNDYx.rp8wq2Gp1cZ2VID7kU4yvbryfKT7BmYUtawMP_Ttd4gg.Upnmqrr7rgvZ9W76f6XqWzc-VI4B5789lkT3jAE-Sd0g.JPEG/KB%EC%B0%A8%EC%B0%A8%EC%B0%A8_%EC%A4%91%EA%B3%A0%EC%B0%A8_%EC%9E%90%EB%8F%99%EC%B0%A8_%EC%9D%B8%EA%B8%B0%EC%A4%91%EA%B3%A0%EC%B0%A8_Top_10_7.JPG?type=w1200
Total Count: 137
Succsessful Count: 132
URL: https://www.theclasshyosung.com/content/dam/south-korea/retailer/retail-kr/tchs/star%20tradein_530x250.jpg.asset.70jduDZGcYwBvlNGRA2t

URL: https://post-phinf.pstatic.net/MjAxOTAxMjhfMjI4/MDAxNTQ4NjQyNTAxMDQ3.CktIxG9WguhEio9NtQ7UVicKihFBN1-KZ60PQSu7Kk8g.IRt3-1wMl_1ml2_tK6Smp2njeDjfh5S8CVZg-xkGcbIg.PNG/K5%EC%A4%91%EA%B3%A0%EC%B0%A8%2C_%EC%B0%A9%ED%95%9C%EC%B0%A8%EC%A7%91_%EA%B5%AC%EB%A7%A4%ED%9B%84%EA%B8%B0_%281%29.PNG?type=w1200
Total Count: 195
Succsessful Count: 188
URL: http://file.mk.co.kr/meet/neds/2017/06/image_readtop_2017_367684_14962951222903327.jpg
Total Count: 196
Succsessful Count: 189
URL: http://image.kmib.co.kr/online_image/2013/0915/130915_18_1.jpg
Total Count: 197
Succsessful Count: 190
URL: http://www.%EC%9D%98%EC%A0%95%EB%B6%80%EC%A4%91%EA%B3%A0%EC%B0%A81004.com/app/dubu_board/docs/imgs/w/xs_w15638660632546_20190723_150219.jpg
can't get img
Total Count: 198
Succsessful Count: 190
URL: http://image.newdaily.co.kr/site/data/img/2019/01/16/2019011600057_0.jpg
Total Count: 199
Succsessful Count: 191
URL: http://img.danawa.com/images/news/images/000100/20180828173236809_G2CEWKA3.jpg
can't get img
Total C

Total Count: 254
Succsessful Count: 245
URL: https://www.car88.kr/data/editor/1902/b23c5bb2b37ceebd7fe582687b396075_1551084997_7607.png
can't get img
Total Count: 255
Succsessful Count: 245
URL: http://image.koreatimes.com/article/2019/04/03/201904031644045c1.gif
Total Count: 256
Succsessful Count: 246
URL: https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=1142563035878295
Total Count: 257
Succsessful Count: 247
URL: http://www.monthlymaritimekorea.com/news/photo/201903/23681_18793_5348.jpg
Total Count: 258
Succsessful Count: 248
URL: http://static.hubzum.zumst.com/hubzum/2019/07/22/11/3957b6cbc2b64a9fac5e2b4918c4e051.png
Total Count: 259
Succsessful Count: 249
URL: http://img.hani.co.kr/imgdb/resize/2012/0604/00304879401_20120604.JPG
Total Count: 260
Succsessful Count: 250
URL: http://image.itdonga.com/files/2015/09/21/001_HF9AAC3.jpg
Total Count: 261
Succsessful Count: 251
URL: https://i.ytimg.com/vi/hQHstzlYENc/maxresdefault.jpg
Total Count: 262
Succsessful Count: 252
UR

Total Count: 15
Succsessful Count: 14
URL: http://cfile215.uf.daum.net/image/995EB43359DC363C168C20
Total Count: 16
Succsessful Count: 15
URL: https://post-phinf.pstatic.net/MjAxNzA1MTFfMzQg/MDAxNDk0NDYyNjI0OTE1.V0NGklqzljgf_eeywRsicSvwWsyWMWFfEi8MZSVDZ4kg.JHO0Ll-Ji9bKsTJapj_oGmJfRqWS-St8JtBo74NWxd4g.JPEG/image_4577154601494462603138.jpg?type=w1200
Total Count: 17
Succsessful Count: 16
URL: https://t1.daumcdn.net/cfile/tistory/9910903E5C74A5E31C
Total Count: 18
Succsessful Count: 17
URL: https://t1.daumcdn.net/cfile/tistory/261DF9455744500410
Total Count: 19
Succsessful Count: 18
URL: https://img.seoul.co.kr/img/upload/2017/04/24/SSI_20170424173516.jpg
Total Count: 20
Succsessful Count: 19
URL: https://t1.daumcdn.net/cfile/tistory/160D2E1E4BD5C8053D
Total Count: 21
Succsessful Count: 20
URL: http://kr.people.com.cn/NMediaFile/2017/0413/FOREIGN201704131037000374946304768.jpg
can't get img
Total Count: 22
Succsessful Count: 20
URL: https://img-s-msn-com.akamaized.net/tenant/amp/entityid/